In [1]:
!pip install transformers

In [14]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer

In [15]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


In [16]:
url = "https://www.alpenpark-neuss.de/fileadmin/_processed_/4/e/csm_Gruppe_Party_2_76e725af0e.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [17]:
inputs = processor(text=["a photo of a hut", "a photo of a party"], images=image, return_tensors="pt", padding=True)

In [18]:
outputs = model(**inputs)

In [19]:
logits_per_image = outputs.logits_per_image

In [20]:
probs = logits_per_image.softmax(dim=1)

In [21]:
probs

tensor([[1.9063e-05, 9.9998e-01]], grad_fn=<SoftmaxBackward>)

In [58]:
import numpy as np


In [4]:
image_urls = ["https://www.alpenpark-neuss.de/fileadmin/_processed_/4/e/csm_Gruppe_Party_2_76e725af0e.jpg",
              "https://thumbs.dreamstime.com/b/chateau-winter-mountains-hut-snow-winter-mountain-landscape-karkonosze-poland-chateau-winter-mountains-137201032.jpg",
              "https://www.lorrainefinneyphotography.com/wp-content/uploads/2018/02/Early-Morning-Fog-1-1.jpg",
              "https://lp-cms-production.imgix.net/2020-11/shutterstockRF_275298065.jpg"]

<table>
    <tbody>
        <tr>
            <td><img src='https://www.alpenpark-neuss.de/fileadmin/_processed_/4/e/csm_Gruppe_Party_2_76e725af0e.jpg' width="400"></td>
            <td><img src='https://thumbs.dreamstime.com/b/chateau-winter-mountains-hut-snow-winter-mountain-landscape-karkonosze-poland-chateau-winter-mountains-137201032.jpg' width="400"></td>
        </tr>
        <tr>
            <td><img src='https://www.lorrainefinneyphotography.com/wp-content/uploads/2018/02/Early-Morning-Fog-1-1.jpg' width="400"></td>
            <td><img src='https://lp-cms-production.imgix.net/2020-11/shutterstockRF_275298065.jpg' width="400"></td>
        </tr>
    </tbody>
</table>

In [27]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer
from scipy.special import softmax

### 1. Get image embeddings

In [14]:
image_embeddings = []

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.
ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


In [22]:
for url in image_urls:
    print(url)
    image = Image.open(requests.get(url, stream=True).raw)
    inputs = processor(images=image, return_tensors="pt")
    image_features = model.get_image_features(**inputs)
    image_embeddings.append(image_features)

https://www.alpenpark-neuss.de/fileadmin/_processed_/4/e/csm_Gruppe_Party_2_76e725af0e.jpg
https://thumbs.dreamstime.com/b/chateau-winter-mountains-hut-snow-winter-mountain-landscape-karkonosze-poland-chateau-winter-mountains-137201032.jpg
https://www.lorrainefinneyphotography.com/wp-content/uploads/2018/02/Early-Morning-Fog-1-1.jpg
https://lp-cms-production.imgix.net/2020-11/shutterstockRF_275298065.jpg


### 2. Get text embeddings

In [65]:
text = ["a photo of a party", 
        "a photo of a winter hut",
        "a photo of fog",
        "a photo of a forest hike",
        "something different"]

inputs = tokenizer(text, padding=True, return_tensors="pt")
text_features = model.get_text_features(**inputs)

### 3. Match image and text embeddings

In [66]:
for image_embedding in image_embeddings:
    res = np.dot(image_embedding.detach().numpy(), text_features.detach().numpy().T)
    res = np.round(softmax(res), 3)

    for idx in range(len(text)):
        print("{:.3f} -> {}".format(res[0][idx], text[idx]))

    print('\n')

1.000 -> a photo of a party
0.000 -> a photo of a winter hut
0.000 -> a photo of fog
0.000 -> a photo of a forest hike
0.000 -> something different


0.002 -> a photo of a party
0.985 -> a photo of a winter hut
0.000 -> a photo of fog
0.000 -> a photo of a forest hike
0.012 -> something different


0.001 -> a photo of a party
0.000 -> a photo of a winter hut
0.965 -> a photo of fog
0.000 -> a photo of a forest hike
0.033 -> something different


0.009 -> a photo of a party
0.000 -> a photo of a winter hut
0.000 -> a photo of fog
0.985 -> a photo of a forest hike
0.006 -> something different


